In [ ]:
import os
import openpyxl
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
folder = r'\path\to\folder'

#### Getting filenames that all have "raw" sheets

In [ ]:
sn_df = pd.DataFrame(columns=['root','filename','sheetname'])
for root,dirs,files in os.walk(folder):
    for f in files:
        try:
            if ('SOTD' in f or 'PLTS' in f) and root.split('\\')[-1].startswith('20'):
                file = root + '\\' + f
                xl = openpyxl.load_workbook(file)
                for sheet in xl.sheetnames:
                    sn_df.loc[len(sn_df)] = [file,f,sheet]
        except Exception as e:
            print('File errored: %s\nError: %s\n'%(file,e))

In [ ]:
sn_df['raw_check'] = sn_df['sheetname'].map(lambda x: True if 'raw' in str(x).lower() else False)

In [ ]:
cols_df = pd.DataFrame(columns=['root','filename','column'])
for row in sn_df[sn_df['raw_check']==True].iterrows():
    root = row[1]['root']
    filename = row[1]['filename']
    sheet = row[1]['sheetname']
    df = pd.read_excel(root,sheet_name=sheet,skiprows=1)
    df = df.drop(df.columns[0:2], axis=1)
    for c in df.columns:
        cols_df.loc[len(cols_df)] = [root,filename,c]

### Reading data from the raw sheets in each excel file
### Loading data into dataframes, making a list of dataframes

In [ ]:
dfs = []
for row in sn_df[sn_df['raw_check']==True].iterrows():
    root = row[1]['root']
    filename = row[1]['filename']
    sheet = row[1]['sheetname']
    df = pd.read_excel(root,sheet_name=sheet,skiprows=1)
    df = df.drop(df.columns[0:2], axis=1)
    df.insert(0,'filename',filename)
    dfs += [df]

### Making a list of the columns that are present in the majority of raw data sheets

In [ ]:
cols = cols_df.groupby(['column'])['column'].count().sort_values()

In [ ]:
columns = [c for c in cols.tail(25).index]
columns = ['filename'] + columns

### Loading all dataframes into one dataframe and exporting to CSV

In [ ]:
full_df = pd.DataFrame(columns=columns)
for d in dfs:
    if 'Material Number P/N' in d.columns:
        d['Material Number'] = d['Material Number P/N']
    else:
        pass
    if 'NRD Date' not in d.columns:
        d['NRD Date'] = None
    if 'PO SCL Delivery Date' not in d.columns:
        d['PO SCL Delivery Date'] = None
    if 'Buyer Name' not in d.columns:
        d['Buyer Name'] = None
    full_df = pd.concat([full_df,d])
full_df.to_csv(r'\path\to\csv\file.csv',index=False)

In [ ]:
full_df.shape[0]

In [ ]:
sum_length = 0
for x in dfs:
    sum_length += x.shape[0]
print(sum_length)